In [17]:
#This code compresses PROCEDURES_ICD9, and DIAGNOSES_ICD9 into single entries per admission (sorted by SEQ_NUM)
#and merges this data with the ADMISSIONS dataframe, providing two separate dataframes with this merged structure.
#One dataframe corresponds to patients diagnosed with specified ICD-9 codes and the other contains the control patients.

import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

ADMISSIONS = pd.read_csv("./ADMISSIONS.csv")
DIAGNOSES_ICD = pd.read_csv("./DIAGNOSES_ICD.csv")
PATIENTS = pd.read_csv("./PATIENTS.csv")
PROCEDURES_ICD = pd.read_csv("./PROCEDURES_ICD.csv")

#Input ICD9 code that you want to look at here:
my_icd9_code = ["44100", "44101", "44102", "44103"] #441 is arotic dissection. Change to 421 for bacterial endocarditis
my_text_code = "AORTIC DISSECTION"

# for bacertial endocarditis
#my_icd9_code = ["4210", "4211", "4219"] #421 is bacterial endocarditis. Change to 441 for aortic dissection
#my_text_code = "BACTERIAL ENDOCARDITIS"

# for Arterial Thromboembolism
#set my_icd9_code to be all diagnoses that start with 444
#my_icd9_code = [4440, 4441, 44421, 44422, 44481, 44489, 4449]
#my_text_code = "ARTERIAL THROMBOEMBOLISM"

# for spinal abcess #324.1
#my_icd9_code = [3241, 32410]
#my_text_code = "SPINAL ABSCESS"

#check what any following numbers would be in the ICD9 code
#need to update to be 441.00, 441.01, 441.02, 441.03

#Returns patients with aortic dissection
AD_SUBJECT_ID = DIAGNOSES_ICD.loc[
    DIAGNOSES_ICD["ICD9_CODE"].astype(str).isin(my_icd9_code),
    "SUBJECT_ID"
].unique() 

print(f"Number of unique patients with the specified ICD-9 code: {len(AD_SUBJECT_ID)}")

"""
Adding filter to remove the text based AD diagnoses that don't have the ICD9 code
"""

# Also grab patients with aortic dissection in admission text
text_ids = ADMISSIONS[
    ADMISSIONS['DIAGNOSIS'].str.contains(my_text_code, na=False)
]['SUBJECT_ID'].unique()

# Combine both sets — these should ALL be excluded from controls
EXCLUDE_FROM_CONTROLS = np.union1d(AD_SUBJECT_ID, text_ids)

# Now build controls excluding both groups
CONTROL_SUBJECT_ID = DIAGNOSES_ICD.loc[
    ~DIAGNOSES_ICD["SUBJECT_ID"].isin(EXCLUDE_FROM_CONTROLS),
    "SUBJECT_ID"
].unique()

"""
Done adding
"""

"""
commenting out as it's now redundant
CONTROL_SUBJECT_ID = DIAGNOSES_ICD.loc[
    ~DIAGNOSES_ICD["SUBJECT_ID"].isin(AD_SUBJECT_ID), 
    "SUBJECT_ID"
].unique()
"""

#Returns the specific admissions where aortic dissection was diagnosed
AD_HADM_ID = DIAGNOSES_ICD.loc[
    DIAGNOSES_ICD["ICD9_CODE"].astype(str).isin(my_icd9_code),
    "HADM_ID"
].unique()


#Identify all diagnoses for patients diagnosed with aortic dissection, including for admissions where they were not diagnosed with AD
PATIENT_DIAGNOSES = DIAGNOSES_ICD[DIAGNOSES_ICD['SUBJECT_ID'].isin(AD_SUBJECT_ID)]

CONTROL_DIAGNOSES = DIAGNOSES_ICD[DIAGNOSES_ICD['SUBJECT_ID'].isin(CONTROL_SUBJECT_ID)]

#Return a new dataframe with all the ICD9 codes for each admission condensed into a single row,col val as a compressed list
PATIENT_DIAGNOSES = (
    PATIENT_DIAGNOSES
    .sort_values(['HADM_ID','SEQ_NUM'])
    .groupby(['SUBJECT_ID','HADM_ID'])['ICD9_CODE']
    .apply(list)
    .reset_index(name='DIAGNOSES')
)

#Return a new dataframe with all the ICD9 codes for each admission condensed into a single row,col val as a compressed list for control patients
CONTROL_DIAGNOSES = (
    CONTROL_DIAGNOSES
    .sort_values(['HADM_ID','SEQ_NUM'])
    .groupby(['SUBJECT_ID','HADM_ID'])['ICD9_CODE']
    .apply(list)
    .reset_index(name='DIAGNOSES')
)

#Remove DIAGNOSES_ICD to conserve memory since we have already filtered for the relevant data
#del DIAGNOSES_ICD

#Return all procedures for patients diagnosed with AD, including for admissions where they were not diagnosed with AD
PATIENT_PROCEDURES = PROCEDURES_ICD[PROCEDURES_ICD['SUBJECT_ID'].isin(AD_SUBJECT_ID)]

#identify all procedures for control patients as well
CONTROL_PROCEDURES = PROCEDURES_ICD[PROCEDURES_ICD['SUBJECT_ID'].isin(CONTROL_SUBJECT_ID)]

#Return a new dataframe with all procedure codes for each admission compressed into a single row,col val as a compressed list
PATIENT_PROCEDURES = (
    PATIENT_PROCEDURES
    .sort_values(['HADM_ID','SEQ_NUM'])
    .groupby(['SUBJECT_ID','HADM_ID'])['ICD9_CODE']
    .apply(list)
    .reset_index(name='PROCEDURE TYPE')
)

#Return a new dataframe with all procedure codes for each admission compressed into a single row,col val as a compressed list for control patients
CONTROL_PROCEDURES = (
    CONTROL_PROCEDURES
    .sort_values(['HADM_ID','SEQ_NUM'])
    .groupby(['SUBJECT_ID','HADM_ID'])['ICD9_CODE']
    .apply(list)
    .reset_index(name='PROCEDURE TYPE')
)

#Remove PROCEDURES_ICD to conserve memory since we have already extracted the relevant rows
#del PROCEDURES_ICD

#Return every admission entry for patients who were diagnosed with AD at some point
PATIENT_ADMISSIONS = ADMISSIONS[ADMISSIONS['SUBJECT_ID'].isin(AD_SUBJECT_ID)]

#pull control group admissions as well
CONTROL_ADMISSIONS = ADMISSIONS[ADMISSIONS['SUBJECT_ID'].isin(CONTROL_SUBJECT_ID)]

#Remove redundant columns from the other filtered dataframes for a cleaner merge
PATIENT_PROCEDURES.drop('SUBJECT_ID',inplace=True,axis=1)
PATIENT_DIAGNOSES.drop('SUBJECT_ID',inplace=True,axis=1)
CONTROL_PROCEDURES.drop('SUBJECT_ID',inplace=True,axis=1)
CONTROL_DIAGNOSES.drop('SUBJECT_ID',inplace=True,axis=1)

#Merge the compressed DFs engineered earlier with admissions so that each admission has lab event, diagnosis, and procedure data
PATIENT_ADMISSIONS_MERGED = PATIENT_ADMISSIONS.merge(PATIENT_DIAGNOSES, on="HADM_ID", how="left") \
            .merge(PATIENT_PROCEDURES, on="HADM_ID", how="left")

CONTROL_ADMISSIONS_MERGED = CONTROL_ADMISSIONS.merge(CONTROL_DIAGNOSES, on="HADM_ID", how="left") \
            .merge(CONTROL_PROCEDURES, on="HADM_ID", how="left")

#Rename columns for clarity since there is a text-based labeling column and the ICD-9 diagnosis column
PATIENT_ADMISSIONS_MERGED = PATIENT_ADMISSIONS_MERGED.rename(columns={"DIAGNOSIS": "DIAGNOSIS (LABEL)","DIAGNOSES": "DIAGNOSIS (ICD_9)"})
CONTROL_ADMISSIONS_MERGED = CONTROL_ADMISSIONS_MERGED.rename(columns={"DIAGNOSIS": "DIAGNOSIS (LABEL)","DIAGNOSES": "DIAGNOSIS (ICD_9)"})

#Drop redundant row
PATIENT_ADMISSIONS_MERGED.drop(['ROW_ID'],inplace=True,axis=1)
CONTROL_ADMISSIONS_MERGED.drop(['ROW_ID'],inplace=True,axis=1)

#Identify the admissions where AD was one of the diagnoses given to the patients, excluding admissions where AD was not diagnosed
#No need to do this for control group
DISEASE_ADMISSIONS = PATIENT_ADMISSIONS_MERGED[PATIENT_ADMISSIONS_MERGED['HADM_ID'].isin(AD_HADM_ID)]
DISEASE_ADMISSIONS = DISEASE_ADMISSIONS.copy()

#Convert ADMITTIME to datetime for processing
DISEASE_ADMISSIONS['ADMITTIME'] = pd.to_datetime(DISEASE_ADMISSIONS["ADMITTIME"], errors="coerce")

#convert to datetime for control group
CONTROL_ADMISSIONS_MERGED['ADMITTIME'] = pd.to_datetime(CONTROL_ADMISSIONS_MERGED["ADMITTIME"], errors="coerce")

#Sort by HADM_ID and ADMITTIME to get a sorted list for processing
DISEASE_ADMISSIONS = DISEASE_ADMISSIONS.sort_values(['HADM_ID','ADMITTIME'])

#sort control group by HADM_ID and ADMITTIME as well
CONTROL_ADMISSIONS_MERGED = CONTROL_ADMISSIONS_MERGED.sort_values(['HADM_ID','ADMITTIME'])

#Identify the earliest admission time in which patients were diagnosed with AD
DISEASE_FIRST_ADMISSIONS = DISEASE_ADMISSIONS.groupby('SUBJECT_ID',as_index=False)['ADMITTIME'].min()

#Rename this column to "Comparator" since it will be used for filtering admissions from after the patient was diagnosed with AD
DISEASE_FIRST_ADMISSIONS = DISEASE_FIRST_ADMISSIONS.rename(columns={"ADMITTIME": "Comparator"})

#Update ADMISSIONS_MERGED so it now contains all admissions for patients who were diagnosed with AD at some point
#Prior and including the admission with their first diagnosis of AD. Admissions after their first diagnosis are excluded
PATIENT_ADMISSIONS_MERGED['ADMITTIME'] = pd.to_datetime(PATIENT_ADMISSIONS_MERGED["ADMITTIME"], errors="coerce")
PATIENT_ADMISSIONS_MERGED = PATIENT_ADMISSIONS_MERGED.merge(DISEASE_FIRST_ADMISSIONS,on='SUBJECT_ID',how="left")
PATIENT_ADMISSIONS_MERGED = PATIENT_ADMISSIONS_MERGED[PATIENT_ADMISSIONS_MERGED['ADMITTIME']<PATIENT_ADMISSIONS_MERGED['Comparator']]

#drop the comparator column now that filtering is done so that the DFs are the same
PATIENT_ADMISSIONS_MERGED = PATIENT_ADMISSIONS_MERGED.drop(['Comparator'],axis=1)

Number of unique patients with the specified ICD-9 code: 343


In [18]:
#Thomas's logic retained below

print(PATIENT_ADMISSIONS_MERGED.columns)
print(PATIENT_ADMISSIONS_MERGED.shape)

print(CONTROL_ADMISSIONS_MERGED.columns)
print(CONTROL_ADMISSIONS_MERGED.shape)

Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS (LABEL)', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'DIAGNOSIS (ICD_9)', 'PROCEDURE TYPE'],
      dtype='str')
(35, 20)
Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS (LABEL)', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'DIAGNOSIS (ICD_9)', 'PROCEDURE TYPE'],
      dtype='str')
(58440, 20)


In [19]:
chunksize = 50000
LAB_EVENTS = pd.read_csv("./LABEVENTS.csv",chunksize=chunksize)
LAB_IDS = pd.read_csv("./Lab_Item_Codes.txt", sep="\t")

PATIENT_LAB_EVENTS2 = []
CONTROL_LAB_EVENTS2 = []

labs_of_interest = [
    'TROPONIN', 'D-DIMER', 'CREATININE', 'BUN', 
    'UREA', 'C-REACTIVE', 'LDH', 'LACTATE DEHYDROGENASE', 
    'BILIRUBIN', 'AST', 'ALT', 
    'WHITE BLOOD', 'WBC', 'LYMPHOCYTE', 'NEUTROPHIL'
]

#create one large string which the string matcher will search through
pattern = '|'.join(labs_of_interest)

#check for any of the strings listed above in the pattern mega-string
lab_ids = LAB_IDS[
    LAB_IDS['Display'].str.contains(pattern, case=False, na=False)
]
lab_ids = lab_ids['Code']

for EVENT in LAB_EVENTS:

    unique_admission_count = len(EVENT['HADM_ID'].unique())
    unique_admission_count += unique_admission_count

    EVENT = EVENT.loc[
        EVENT['ITEMID'].isin(lab_ids)
    ]
    
    #Identify all patient labs for patients who were diagnosed with AD at any point. Do not include labs taken during admissions following their first diagnosis
    PATIENT_LAB_EVENTS = EVENT[EVENT['HADM_ID'].isin(PATIENT_ADMISSIONS_MERGED['HADM_ID'])]
    
    #pull control group lab events as well
    CONTROL_LAB_EVENTS = EVENT[EVENT['HADM_ID'].isin(CONTROL_ADMISSIONS_MERGED['HADM_ID'])]
    
    #Remove redundant rows
    PATIENT_LAB_EVENTS = PATIENT_LAB_EVENTS.drop(['ROW_ID','VALUE'],axis=1)
    CONTROL_LAB_EVENTS = CONTROL_LAB_EVENTS.drop(['ROW_ID','VALUE'],axis=1)
    
    #Convert CHARTTIME to a datetime for sorting
    PATIENT_LAB_EVENTS['CHARTTIME'] = pd.to_datetime(PATIENT_LAB_EVENTS["CHARTTIME"], errors="coerce")
    CONTROL_LAB_EVENTS['CHARTTIME'] = pd.to_datetime(CONTROL_LAB_EVENTS["CHARTTIME"], errors="coerce")
    
    #Only return values that are not na.
    PATIENT_LAB_EVENTS = PATIENT_LAB_EVENTS[PATIENT_LAB_EVENTS['HADM_ID'].notna()]
    CONTROL_LAB_EVENTS = CONTROL_LAB_EVENTS[CONTROL_LAB_EVENTS['HADM_ID'].notna()]

    PATIENT_LAB_EVENTS2.append(PATIENT_LAB_EVENTS)
    CONTROL_LAB_EVENTS2.append(CONTROL_LAB_EVENTS)
    
#Remove LAB_EVENTS to conserve lots of memory since we already have extracted the necessary data
del LAB_EVENTS

#Construct final labs DFs through concatenation
FINAL_PATIENT_LABS = pd.concat(PATIENT_LAB_EVENTS2)
FINAL_CONTROL_LABS = pd.concat(CONTROL_LAB_EVENTS2)

print(unique_admission_count)

136


In [20]:
def apply_event_index_filter(PATIENT_ADMISSIONS_MERGED, CONTROL_ADMISSIONS_MERGED, AD_HADM_ID):

    PATIENT_ADMISSIONS_MERGED = PATIENT_ADMISSIONS_MERGED.copy()
    CONTROL_ADMISSIONS_MERGED = CONTROL_ADMISSIONS_MERGED.copy()

    PATIENT_ADMISSIONS_MERGED["ADMITTIME"] = pd.to_datetime(
        PATIENT_ADMISSIONS_MERGED["ADMITTIME"], errors="coerce"
    )

    CONTROL_ADMISSIONS_MERGED["ADMITTIME"] = pd.to_datetime(
        CONTROL_ADMISSIONS_MERGED["ADMITTIME"], errors="coerce"
    )

    PATIENT_ADMISSIONS_MERGED = PATIENT_ADMISSIONS_MERGED.sort_values(
        ["SUBJECT_ID", "ADMITTIME"]
    )

    """
    PATIENT_ADMISSIONS_MERGED["PATIENT_ADMISSION_INDEX"] = (
        PATIENT_ADMISSIONS_MERGED
        .groupby("SUBJECT_ID")
        .cumcount() + 1
    )
    """

    DISEASE_ADMISSIONS = PATIENT_ADMISSIONS_MERGED[
        PATIENT_ADMISSIONS_MERGED["HADM_ID"].isin(AD_HADM_ID)
    ].copy()

    DISEASE_ADMISSIONS = DISEASE_ADMISSIONS.sort_values(
        ["SUBJECT_ID", "ADMITTIME"]
    )

    DISEASE_FIRST_ADMISSIONS = (
        DISEASE_ADMISSIONS
        .groupby("SUBJECT_ID", as_index=False)["ADMITTIME"]
        .min()
        .rename(columns={"ADMITTIME": "Comparator"})
    )

    PATIENT_ADMISSIONS_MERGED = PATIENT_ADMISSIONS_MERGED.merge(
        DISEASE_FIRST_ADMISSIONS,
        on="SUBJECT_ID",
        how="left"
    )

    PATIENT_ADMISSIONS_MERGED = PATIENT_ADMISSIONS_MERGED[
        (PATIENT_ADMISSIONS_MERGED["Comparator"].isna()) |
        (PATIENT_ADMISSIONS_MERGED["ADMITTIME"] <= PATIENT_ADMISSIONS_MERGED["Comparator"])
    ]

    PATIENT_ADMISSIONS_MERGED.loc[PATIENT_ADMISSIONS_MERGED["Comparator"].notna(), "PATIENT_ADMISSION_INDEX"] = (
        PATIENT_ADMISSIONS_MERGED[PATIENT_ADMISSIONS_MERGED["Comparator"].notna()]
        .groupby("SUBJECT_ID")["ADMITTIME"]
        .rank(method="first", ascending=False)
        .astype(int) - 1
    )
    
    PATIENT_ADMISSIONS_MERGED["PATIENT_ADMISSION_INDEX"] = PATIENT_ADMISSIONS_MERGED["PATIENT_ADMISSION_INDEX"].astype("float")

    PATIENT_ADMISSIONS_MERGED = PATIENT_ADMISSIONS_MERGED.drop(columns=["Comparator"])

    # Keep control DF columns consistent
    CONTROL_ADMISSIONS_MERGED["ADMISSION_INDEX_PER_PATIENT"] = np.nan

    return PATIENT_ADMISSIONS_MERGED, CONTROL_ADMISSIONS_MERGED

In [21]:
PATIENT_ADMISSIONS_MERGED, CONTROL_ADMISSIONS_MERGED = apply_event_index_filter(PATIENT_ADMISSIONS_MERGED, CONTROL_ADMISSIONS_MERGED, AD_HADM_ID)

In [22]:
#display unique ICD9 codes in the filtered patient admissions
#print(PATIENT_ADMISSIONS_MERGED['DIAGNOSIS (ICD_9)'].explode().unique())
#print(CONTROL_ADMISSIONS_MERGED['DIAGNOSIS (ICD_9)'].explode().unique())

print(PATIENT_ADMISSIONS_MERGED.columns)
print(PATIENT_ADMISSIONS_MERGED.shape)
print(CONTROL_ADMISSIONS_MERGED.columns)
print(CONTROL_ADMISSIONS_MERGED.shape)

Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS (LABEL)', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'DIAGNOSIS (ICD_9)', 'PROCEDURE TYPE',
       'PATIENT_ADMISSION_INDEX'],
      dtype='str')
(35, 21)
Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS (LABEL)', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'DIAGNOSIS (ICD_9)', 'PROCEDURE TYPE',
       'ADMISSION_INDEX_PER_PATIENT'],
      dtype='str')
(58440, 21)


In [23]:
PATIENT_ADMISSIONS_MERGED.to_csv(r"C:\BIOE5860_Data\PATIENT_ADMISSIONS_MERGED.csv", index=False)
CONTROL_ADMISSIONS_MERGED.to_csv(r"C:\BIOE5860_Data\CONTROL_ADMISSIONS_MERGED.csv", index=False)

In [24]:
#this script finds all patients who have 'AORTIC DISSECTION' in their ADMISSIONS DIAGNOSIS text field
#but do not have the corresponding ICD9 code in DIAGNOSES_ICD
#my_icd9_code = [whatever yours is]
 
aortic_ids = ADMISSIONS[ADMISSIONS['DIAGNOSIS'].str.contains('AORTIC DISSECTION', na=False)]['SUBJECT_ID'].unique()
 
"""
AD_SUBJECT_ID = DIAGNOSES_ICD.loc[
    DIAGNOSES_ICD["ICD9_CODE"].astype(str).isin(my_icd9_code),
    "SUBJECT_ID"
].unique()
"""
 
#aortic_patients = PATIENT_DIAGNOSES[PATIENT_DIAGNOSES['SUBJECT_ID']].unique()
 
#find patients who have AORTIC DISSECTION text but don't have the ICD9 code from my_icd9_code

difference = set(aortic_ids) - set(AD_SUBJECT_ID)
reamining = len(set(difference) & set(CONTROL_SUBJECT_ID)) #if this is 0 then all were successfully removed in first cell
print(f'Number of patients with AD text but no ICD9 code remaining: {reamining}')


#print(len(difference))
#print(difference)
 
#remove all subject IDs found above from CONTROL_SUBJECT_ID since we don't know if they are actually controls 
#or if they are patients with AD who just don't have the correct ICD9 code in the DIAGNOSES_ICD file
#CONTROL_SUBJECT_ID = CONTROL_SUBJECT_ID[~np.isin(CONTROL_SUBJECT_ID, list(difference))]

#PATIENT_ADMISSIONS_MERGED.to_csv(r"C:\BIOE5860_Data\PATIENT_ADMISSIONS_MERGED.csv", index=False)
#CONTROL_ADMISSIONS_MERGED.to_csv(r"C:\BIOE5860_Data\CONTROL_ADMISSIONS_MERGED.csv", index=False)

print(PATIENT_ADMISSIONS_MERGED.columns)
print(PATIENT_ADMISSIONS_MERGED.shape)
print(CONTROL_ADMISSIONS_MERGED.columns)
print(CONTROL_ADMISSIONS_MERGED.shape)

Number of patients with AD text but no ICD9 code remaining: 0
Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS (LABEL)', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'DIAGNOSIS (ICD_9)', 'PROCEDURE TYPE',
       'PATIENT_ADMISSION_INDEX'],
      dtype='str')
(35, 21)
Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS (LABEL)', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'DIAGNOSIS (ICD_9)', 'PROCEDURE TYPE',
       'ADMISSION_INDEX_PER_PATIENT'],
      dtype='str')
(58440, 21)


In [25]:
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.impute import SimpleImputer

MIN_FREQ = 10  # Minimum frequency threshold for one-hot encoding

"""
NOTE, do we want this minimum frequency? It's there for dimensionality reduction/memory but
we may want to keep more items for the sake of model performance and then do dimensionality reduction later if needed.
We can also look at the distribution of features and decide on a case by case basis if we want to exclude certain ones based on frequency.
"""

# Tag each group
PATIENT_ADMISSIONS_MERGED['LABEL'] = 1
CONTROL_ADMISSIONS_MERGED['LABEL'] = 0

def safe_parse_list(val):
    """Safely parse a list stored as string or return as-is if already a list."""
    if isinstance(val, list):
        return val
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):
            return []
    return []

# Ensure datetime
PATIENT_ADMISSIONS_MERGED['ADMITTIME'] = pd.to_datetime(
    PATIENT_ADMISSIONS_MERGED['ADMITTIME'], errors='coerce'
)
CONTROL_ADMISSIONS_MERGED['ADMITTIME'] = pd.to_datetime(
    CONTROL_ADMISSIONS_MERGED['ADMITTIME'], errors='coerce'
)

# Parse list columns
for df in [PATIENT_ADMISSIONS_MERGED, CONTROL_ADMISSIONS_MERGED]:
    df['DIAG_LIST'] = df['DIAGNOSIS (ICD_9)'].apply(safe_parse_list).apply(
        lambda codes: [str(c).strip() for c in codes]
    )
    df['PROC_LIST'] = df['PROCEDURE TYPE'].apply(safe_parse_list).apply(
        lambda codes: [str(c).strip() for c in codes]
    )

# For controls, assign an admission index (reverse chronological, same logic as patients)
# so that index 0 = their most recent admission
CONTROL_ADMISSIONS_MERGED = CONTROL_ADMISSIONS_MERGED.sort_values(
    ['SUBJECT_ID', 'ADMITTIME']
)
CONTROL_ADMISSIONS_MERGED['PATIENT_ADMISSION_INDEX'] = (
    CONTROL_ADMISSIONS_MERGED
    .groupby('SUBJECT_ID')['ADMITTIME']
    .rank(method='first', ascending=False)
    .astype(int) - 1
)

print(f"Patient admissions: {PATIENT_ADMISSIONS_MERGED.shape[0]}")
print(f"Control admissions: {CONTROL_ADMISSIONS_MERGED.shape[0]}")

Patient admissions: 35
Control admissions: 58440


In [26]:
# Calculate how many unique subjects in the patient group has more than one admission overall
patient_admission_counts = PATIENT_ADMISSIONS_MERGED.groupby('SUBJECT_ID').size()
patients_with_multiple_admissions = patient_admission_counts[patient_admission_counts > 1].count()
print(f"Number of unique patients with more than one admission: {patients_with_multiple_admissions}")

Number of unique patients with more than one admission: 5


In [27]:
def build_temporal_code_properties(df, code_col, prefix):
    """
    For each patient, build:
      - {prefix}_{code}_ever: 1 if code appears in any admission
      - {prefix}_{code}_at_dx: 1 if code appears at index 0 (diagnosis/reference admission)
      - {prefix}_{code}_first_idx: earliest admission index where code appeared (0 = diagnosis admission, higher = further back)
    """
    rows = []
    for subj, grp in df.groupby('SUBJECT_ID'):
        ever_codes = set()
        at_dx_codes = set()
        first_idx = {}

        for _, admission in grp.iterrows():
            idx = admission['PATIENT_ADMISSION_INDEX']
            codes = admission[code_col]

            for c in codes:
                ever_codes.add(c)
                if idx == 0:
                    at_dx_codes.add(c)
                # Track the highest index (furthest back) where code first appeared
                if c not in first_idx or idx > first_idx[c]:
                    first_idx[c] = idx

        rows.append({
            'SUBJECT_ID': subj,
            '_ever': ever_codes,
            '_at_dx': at_dx_codes,
            '_first_idx': first_idx
        })

    return rows

# Remove AD diagnosis codes from the list of codes we are building properties for
# since those are the reference point and we want to look at other codes in relation to that
# Remove target-leaking codes — these ARE the diagnosis we're predicting
LEAK_DIAG_CODES = my_icd9_code

for df in [PATIENT_ADMISSIONS_MERGED, CONTROL_ADMISSIONS_MERGED]:
    df['DIAG_LIST'] = df['DIAG_LIST'].apply(
        lambda codes: [c for c in codes if c not in LEAK_DIAG_CODES]
    )

# Now build temporal properties for diagnoses and procedures
print("Building temporal diagnosis properties for patients...")
patient_diag_rows = build_temporal_code_properties(
    PATIENT_ADMISSIONS_MERGED, 'DIAG_LIST', 'DIAG'
)

print("Building temporal diagnosis properties for controls...")
control_diag_rows = build_temporal_code_properties(
    CONTROL_ADMISSIONS_MERGED, 'DIAG_LIST', 'DIAG'
)

print("Building temporal procedure properties for patients...")
patient_proc_rows = build_temporal_code_properties(
    PATIENT_ADMISSIONS_MERGED, 'PROC_LIST', 'PROC'
)

print("Building temporal procedure properties for controls...")
control_proc_rows = build_temporal_code_properties(
    CONTROL_ADMISSIONS_MERGED, 'PROC_LIST', 'PROC'
)

Building temporal diagnosis properties for patients...
Building temporal diagnosis properties for controls...
Building temporal procedure properties for patients...
Building temporal procedure properties for controls...


In [ ]:
def temporal_rows_to_df(rows, prefix):
    """Convert the temporal dicts (diagnoses/procedures/labs) into a wide dataframe."""
    records = []
    for r in rows:
        rec = {'SUBJECT_ID': r['SUBJECT_ID']}
        for c in r['_ever']:
            rec[f'{prefix}_{c}_ever'] = 1
        for c in r['_at_dx']:
            rec[f'{prefix}_{c}_at_dx'] = 1
        for c, idx in r['_first_idx'].items():
            rec[f'{prefix}_{c}_first_idx'] = idx
        records.append(rec)

    df = pd.DataFrame(records).set_index('SUBJECT_ID')

    # Fill NaN for _ever and _at_dx columns with 0
    ever_cols = [c for c in df.columns if c.endswith('_ever')]
    at_dx_cols = [c for c in df.columns if c.endswith('_at_dx')]
    df[ever_cols] = df[ever_cols].fillna(0).astype(int)
    df[at_dx_cols] = df[at_dx_cols].fillna(0).astype(int)

    # _first_idx stays NaN if the code was never seen (will be filled later)
    return df

# Build diagnosis DFs
patient_diag_df = temporal_rows_to_df(patient_diag_rows, 'DIAG')
control_diag_df = temporal_rows_to_df(control_diag_rows, 'DIAG')

# Build procedure DFs
patient_proc_df = temporal_rows_to_df(patient_proc_rows, 'PROC')
control_proc_df = temporal_rows_to_df(control_proc_rows, 'PROC')

# Display some stats about the resulting dataframes
n_patients = len(patient_diag_df)
n_controls = len(control_diag_df)
control_subjects_all = control_diag_df.index.values

print(f"Patient subjects: {n_patients}")
print(f"Control subjects (all): {n_controls}")
print(f"Imbalance ratio: 1:{n_controls // n_patients}")

# Combine patient + control for diagnoses
diag_combined = pd.concat([patient_diag_df, control_diag_df], axis=0).fillna(0)

# Combine patient + control for procedures
proc_combined = pd.concat([patient_proc_df, control_proc_df], axis=0).fillna(0)

# Apply frequency filter to _ever columns (overall code frequency)
diag_ever_cols = [c for c in diag_combined.columns if c.endswith('_ever')]
diag_freq = diag_combined[diag_ever_cols].sum(axis=0)
frequent_diag_codes = [c.replace('_ever', '') for c in diag_ever_cols if diag_freq[c] >= MIN_FREQ]

# Keep only columns for frequent codes with all 3 types (_ever, _at_dx, _first_idx)
diag_keep = []
for code_prefix in frequent_diag_codes:
    for suffix in ['_ever', '_at_dx', '_first_idx']:
        col = f"{code_prefix}{suffix}"
        if col in diag_combined.columns:
            diag_keep.append(col)
diag_combined = diag_combined[diag_keep]

# Same for procedures
proc_ever_cols = [c for c in proc_combined.columns if c.endswith('_ever')]
proc_freq = proc_combined[proc_ever_cols].sum(axis=0)
frequent_proc_codes = [c.replace('_ever', '') for c in proc_ever_cols if proc_freq[c] >= MIN_FREQ]

proc_keep = []
for code_prefix in frequent_proc_codes:
    for suffix in ['_ever', '_at_dx', '_first_idx']:
        col = f"{code_prefix}{suffix}"
        if col in proc_combined.columns:
            proc_keep.append(col)
proc_combined = proc_combined[proc_keep]

print(f"Diagnosis items (3 per code, freq >= {MIN_FREQ}): {diag_combined.shape[1]}")
print(f"Procedure items (3 per code, freq >= {MIN_FREQ}): {proc_combined.shape[1]}")

In [ ]:
#Build a demographics reference from the admissions data (one row per patient)
def get_demographics(df):
    """Pull demographics from the most recent admission (index 0)."""
    idx0 = df[df['PATIENT_ADMISSION_INDEX'] == 0].copy()
    if len(idx0) == 0:
        idx0 = df.sort_values('ADMITTIME', ascending=False).head(1).copy()
    return idx0.iloc[0]

patient_demo = (
    PATIENT_ADMISSIONS_MERGED
    .groupby('SUBJECT_ID')
    .apply(get_demographics)
    [['INSURANCE', 'ETHNICITY', 'MARITAL_STATUS']]
)

control_demo = (
    CONTROL_ADMISSIONS_MERGED
    .groupby('SUBJECT_ID')
    .apply(get_demographics)
    [['INSURANCE', 'ETHNICITY', 'MARITAL_STATUS']]
)

# Admission type properties ie did patient ever have each type?
def get_admission_types(df):
    return df.groupby('SUBJECT_ID')['ADMISSION_TYPE'].apply(
        lambda x: list(x.unique())
    )

patient_admit_types = get_admission_types(PATIENT_ADMISSIONS_MERGED)
control_admit_types = get_admission_types(CONTROL_ADMISSIONS_MERGED)

# Number of admissions per patient
patient_num_admits = PATIENT_ADMISSIONS_MERGED.groupby('SUBJECT_ID')['HADM_ID'].nunique()
control_num_admits = (
    CONTROL_ADMISSIONS_MERGED
    .groupby('SUBJECT_ID')['HADM_ID'].nunique()
)

# Hospital expire flag (max across admissions)
patient_expire = PATIENT_ADMISSIONS_MERGED.groupby('SUBJECT_ID')['HOSPITAL_EXPIRE_FLAG'].max()
control_expire = (
    CONTROL_ADMISSIONS_MERGED
    .groupby('SUBJECT_ID')['HOSPITAL_EXPIRE_FLAG'].max()
)

# Combine demographics
demo_combined = pd.concat([patient_demo, control_demo], axis=0)
demo_combined = demo_combined.fillna('UNKNOWN')

# One-hot encode categorical demographics
demo_onehot = pd.get_dummies(demo_combined, prefix_sep='_', dtype=int)

# One-hot encode admission types
admit_types_all = pd.concat([patient_admit_types, control_admit_types])
mlb_admit = MultiLabelBinarizer()
admit_encoded = mlb_admit.fit_transform(admit_types_all)
admit_cols = [f"ADMTYPE_{c}" for c in mlb_admit.classes_]
admit_df = pd.DataFrame(admit_encoded, columns=admit_cols, index=admit_types_all.index)

# Add numeric information about number of admissions and hospital expire flag
num_admits_all = pd.concat([patient_num_admits, control_num_admits]).rename('NUM_ADMISSIONS')
expire_all = pd.concat([patient_expire, control_expire]).rename('HOSPITAL_EXPIRE_FLAG')

demo_final = demo_onehot.join(admit_df).join(num_admits_all).join(expire_all)

# Filter rare categories
demo_final = demo_final.loc[:, demo_final.sum(axis=0) >= MIN_FREQ]

print(f"Demographic items: {demo_final.shape[1]}")

In [ ]:
ALL_LABS = pd.concat([FINAL_PATIENT_LABS, FINAL_CONTROL_LABS], ignore_index=True)

# Build HADM_ID -> SUBJECT_ID mapping from both admission tables
hadm_to_subject = {}
for _, row in PATIENT_ADMISSIONS_MERGED[['HADM_ID', 'SUBJECT_ID']].iterrows():
    hadm_to_subject[row['HADM_ID']] = row['SUBJECT_ID']
for _, row in CONTROL_ADMISSIONS_MERGED[['HADM_ID', 'SUBJECT_ID']].iterrows():
    hadm_to_subject[row['HADM_ID']] = row['SUBJECT_ID']

# Build HADM_ID -> PATIENT_ADMISSION_INDEX mapping
hadm_to_idx = {}
for _, row in PATIENT_ADMISSIONS_MERGED[['HADM_ID', 'PATIENT_ADMISSION_INDEX']].iterrows():
    hadm_to_idx[row['HADM_ID']] = row['PATIENT_ADMISSION_INDEX']
for _, row in CONTROL_ADMISSIONS_MERGED[['HADM_ID', 'PATIENT_ADMISSION_INDEX']].iterrows():
    hadm_to_idx[row['HADM_ID']] = row['PATIENT_ADMISSION_INDEX']

ALL_LABS['SUBJECT_ID_MAPPED'] = ALL_LABS['HADM_ID'].map(hadm_to_subject)
ALL_LABS['ADMISSION_INDEX'] = ALL_LABS['HADM_ID'].map(hadm_to_idx)

# Keep only labs for subjects in our combined set
all_subjects = set(diag_combined.index)
ALL_LABS = ALL_LABS[ALL_LABS['SUBJECT_ID_MAPPED'].isin(all_subjects)]

ALL_LABS['VALUENUM'] = pd.to_numeric(ALL_LABS['VALUENUM'], errors='coerce')
ALL_LABS = ALL_LABS.dropna(subset=['VALUENUM'])

print(f"Total lab rows after filtering: {len(ALL_LABS)}")

# Overall aggregation (across all admissions)
lab_overall = ALL_LABS.groupby(['SUBJECT_ID_MAPPED', 'ITEMID'])['VALUENUM'].agg(
    ['mean', 'min', 'max']
).reset_index()

ALL_LABS_sorted = ALL_LABS.sort_values(['SUBJECT_ID_MAPPED', 'ITEMID', 'CHARTTIME'])
lab_last = (
    ALL_LABS_sorted
    .groupby(['SUBJECT_ID_MAPPED', 'ITEMID'])['VALUENUM']
    .last()
    .reset_index()
)
lab_last.columns = ['SUBJECT_ID_MAPPED', 'ITEMID', 'last']

lab_overall = lab_overall.merge(lab_last, on=['SUBJECT_ID_MAPPED', 'ITEMID'], how='left')

lab_overall_pivot = lab_overall.pivot_table(
    index='SUBJECT_ID_MAPPED',
    columns='ITEMID',
    values=['mean', 'min', 'max', 'last']
)
lab_overall_pivot.columns = [f"LAB_{int(item)}_{stat}" for stat, item in lab_overall_pivot.columns]

# Index-0 specific labs (values at the diagnosis/reference admission)
labs_at_dx = ALL_LABS[ALL_LABS['ADMISSION_INDEX'] == 0].copy()

lab_dx_agg = labs_at_dx.groupby(['SUBJECT_ID_MAPPED', 'ITEMID'])['VALUENUM'].agg(
    ['mean', 'min', 'max']
).reset_index()

labs_at_dx_sorted = labs_at_dx.sort_values(['SUBJECT_ID_MAPPED', 'ITEMID', 'CHARTTIME'])
lab_dx_last = (
    labs_at_dx_sorted
    .groupby(['SUBJECT_ID_MAPPED', 'ITEMID'])['VALUENUM']
    .last()
    .reset_index()
)
lab_dx_last.columns = ['SUBJECT_ID_MAPPED', 'ITEMID', 'last']

lab_dx_agg = lab_dx_agg.merge(lab_dx_last, on=['SUBJECT_ID_MAPPED', 'ITEMID'], how='left')

lab_dx_pivot = lab_dx_agg.pivot_table(
    index='SUBJECT_ID_MAPPED',
    columns='ITEMID',
    values=['mean', 'min', 'max', 'last']
)
lab_dx_pivot.columns = [f"LAB_{int(item)}_dx_{stat}" for stat, item in lab_dx_pivot.columns]

# Deltas: index-0 mean minus overall mean
# This captures how abnormal a patient's labs were at presentation compared to their own historical baseline

# Get overall mean per (patient, lab)
overall_mean = lab_overall[['SUBJECT_ID_MAPPED', 'ITEMID', 'mean']].copy()
overall_mean.columns = ['SUBJECT_ID_MAPPED', 'ITEMID', 'overall_mean']

# Get index-0 mean per (patient, lab)
dx_mean = lab_dx_agg[['SUBJECT_ID_MAPPED', 'ITEMID', 'mean']].copy()
dx_mean.columns = ['SUBJECT_ID_MAPPED', 'ITEMID', 'dx_mean']

# Merge and compute delta
delta = overall_mean.merge(dx_mean, on=['SUBJECT_ID_MAPPED', 'ITEMID'], how='inner')
delta['delta'] = delta['dx_mean'] - delta['overall_mean']

# Pivot to wide format
delta_pivot = delta.pivot_table(
    index='SUBJECT_ID_MAPPED',
    columns='ITEMID',
    values='delta'
)
delta_pivot.columns = [f"LAB_{int(item)}_delta" for item in delta_pivot.columns]

# Merge overall + dx-specific + delta labs
lab_all = lab_overall_pivot.join(lab_dx_pivot, how='outer').join(delta_pivot, how='outer')
lab_all.index.name = 'SUBJECT_ID'

print(f"Labs (overall + at-diagnosis + delta): {lab_all.shape[1]}")
print(f"  Overall: {lab_overall_pivot.shape[1]}")
print(f"  At-diagnosis: {lab_dx_pivot.shape[1]}")
print(f"  Delta: {delta_pivot.shape[1]}")

In [ ]:
# Create label series
patient_labels = pd.Series(1, index=patient_diag_df.index, name='LABEL')
control_labels = pd.Series(0, index=control_diag_df.index, name='LABEL')
labels = pd.concat([patient_labels, control_labels])

# Join everything on SUBJECT_ID index
final_matrix = pd.DataFrame(labels)
final_matrix = final_matrix.join(diag_combined, how='left')
final_matrix = final_matrix.join(proc_combined, how='left')
final_matrix = final_matrix.join(demo_final, how='left')
final_matrix = final_matrix.join(lab_all, how='left')

# Reset index so SUBJECT_ID becomes a column
final_matrix = final_matrix.reset_index().rename(columns={'index': 'SUBJECT_ID'})

print(f"Final matrix shape: {final_matrix.shape}")
print(f"  Diagnoses: {diag_combined.shape[1]}")
print(f"  Procedures: {proc_combined.shape[1]}")
print(f"  Demographics: {demo_final.shape[1]}")
print(f"  Labs: {lab_all.shape[1]}")

In [ ]:
#Find number of missing values per column
missing_values = final_matrix.isnull().sum()
print("Missing values per column:")
print(missing_values[missing_values > 0])

In [ ]:
# Identify columns (everything except SUBJECT_ID and LABEL)
final_cols = [c for c in final_matrix.columns if c not in ['SUBJECT_ID', 'LABEL']]

# Separate numeric types for imputation
# _first_idx and lab columns are numeric so impute with median
# _ever, _at_dx, and one-hot columns are binary so fill with 0
binary_cols = [c for c in final_cols if c.endswith('_ever') or c.endswith('_at_dx')
               or c.startswith('ADMTYPE_') or c.startswith('INSURANCE_')
               or c.startswith('ETHNICITY_') or c.startswith('MARITAL_STATUS_')]

idx_cols = [c for c in final_cols if c.endswith('_first_idx')]

lab_cols = [c for c in final_cols if c.startswith('LAB_')]

numeric_cols = ['NUM_ADMISSIONS', 'HOSPITAL_EXPIRE_FLAG']

# Fill binary columns with 0
final_matrix[binary_cols] = final_matrix[binary_cols].fillna(0).astype(int)

# Fill _first_idx with -1 (code never seen)
final_matrix[idx_cols] = final_matrix[idx_cols].fillna(-1)

# Fill numeric columns with 0
for c in numeric_cols:
    if c in final_matrix.columns:
        final_matrix[c] = final_matrix[c].fillna(0)

# Impute lab columns with median
if lab_cols:
    imputer = SimpleImputer(strategy='median')
    final_matrix[lab_cols] = imputer.fit_transform(final_matrix[lab_cols])

# Verify no NaNs remain
nan_count = final_matrix[final_cols].isna().sum().sum()
print(f"Remaining NaN's: {nan_count}")

# Save
#final_matrix.to_csv(r"C:\BIOE5860_Data\MODEL_READY_MATRIX.csv", index=False)
final_matrix.to_parquet("./MODEL_READY_MATRIX.parquet", index=False)

print(f"\nSaved MODEL_READY_MATRIX.parquet")
print(f"  Rows: {final_matrix.shape[0]}")
print(f"  Columns: {final_matrix.shape[1]}")
print(f"  Label=1 (AD): {(final_matrix['LABEL']==1).sum()}")
print(f"  Label=0 (Control): {(final_matrix['LABEL']==0).sum()}")

In [ ]:
print("Final matrix columns:")
for i, col in enumerate(final_matrix.columns.tolist()):
    print(f"  {i}: {col}")
print(f"\nTotal: {len(final_cols)}")
print(f"\nFirst 5 rows:")
final_matrix.head()